# Documentation for the creation and usage of the heat pump library (hplib)

This documentation covers the database preperation, validation and usage for simulation. If you're only interested in using hplib for simulation purpose, you should have a look into chapter [4. How to simulate](#how-to-simulate).

1. [Definitions](#definitions)
2. [Database preperation](#database-preparation)
3. [Work with database](#work-with-database)
    1. [Load database](#load-database)
    2. [Load specific model](#load-specific-model)
    3. [Load generic model](#load-generic-model)
4. [How to simulate](#how-to-simulate)
    1. [Simulate on time step](#simulate-one-timestep)
    2. [Simulate a time series](#simulate-a-timeseries)
5. [Example heat pump](#example-heat-pump)
6. [Validation](#validation)
    1. [Air/Water | on/off](#air-water-onoff)
    2. [Brine/Water | on/off](#brine-water-onoff)
    3. [Water/Water | on/off](#water-water-onoff)
    4. [Air/Water | regulated](#air-water-regulated)
        1. [Heating](#air-water-regulated-heating)
        2. [Cooling](#air-water-regulated-cooling)
    5. [Brine/Water | regulated](#brine-water-regulated)
7. [Conclusion](#conclusion)
8. [Additional functions](#additional-functions)

In [1]:
os.chdir(r'../hplib')
import hplib as hpl
import hplib_database as db
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy
warnings.filterwarnings("ignore")

In [2]:
Modul = []
Manufacturer = []
Date = []
Refrigerant = []
Mass = []
Poff = []
Psb = []
Prated = []
SPLindoor = []
SPLoutdoor = []
Type = []
Climate = []
Guideline = []
T_in = []
T_out = []
P_th = []
COP = []
df = pd.DataFrame()
os.chdir('../')
root = os.getcwd()
Scanordner = (root+'/input/txt_new')
os.chdir(Scanordner)
Scan = os.scandir(os.getcwd())
with Scan as dir1:
    for file in dir1:
        with open(file, 'r', encoding='utf-8') as f:
            contents = f.readlines()
            date = 'NaN'
            modul = 'NaN'
            prated_low = 'NaN'
            prated_medium = 'NaN'
            heatpumpType = 'NaN'
            refrigerant = 'NaN'
            splindoor_low = 'NaN'
            splindoor_medium = 'NaN'
            sploutdoor_low = 'NaN'
            sploutdoor_medium = 'NaN'
            poff = 'NaN'
            climate = 'NaN'
            NumberOfTestsPerNorm = []
            NumberOfTestsPerModule = []
            i = 1  # indicator for the line wich is read
            d = 0  # indicator if only medium Temperature is given
            p = 0  # -15° yes or no
            date = contents[1]
            date = date[61:]
            if (date == '17 Dec 2020\n'):
                date = '17.12.2020\n'
            elif (date == '18 Dec 2020\n'):
                date = '18.12.2020\n'
            elif (date.startswith('5 Mar 2021')):
                date = '05.03.2021\n'
            elif (date.startswith('15 Feb 2021')):
                date = '15.02.2021\n'
            elif (date.startswith('22 Feb 2021')):
                date = '22.02.2021\n'
            elif (date.startswith('18 Mar 2022')):
                date = '18.03.2022\n'
            if date.startswith('3x400V 50Hz'):
                print(file)
            for lines in contents:
                i = i + 1
                if (lines.startswith('Name\n') == 1):
                    manufacturer = (contents[i])
                    if (manufacturer.find('(') > 0):
                        manufacturer = manufacturer.split('(', 1)[1].split('\n')[0]
                    if manufacturer.endswith('GmbH\n'):
                        manufacturer = manufacturer[:-5]
                    if manufacturer.endswith('S.p.A.\n'):
                        manufacturer = manufacturer[:-6]
                    if manufacturer.endswith('s.p.a.\n'):
                        manufacturer = manufacturer[:-6]
                    if manufacturer.endswith('S.p.A\n'):
                        manufacturer = manufacturer[:-5]
                    if manufacturer.endswith('S.L.U.\n'):
                        manufacturer = manufacturer[:-6]
                    if manufacturer.endswith('s.r.o.\n'):
                        manufacturer = manufacturer[:-6]
                    if manufacturer.endswith('S.A.\n'):
                        manufacturer = manufacturer[:-4]
                    if manufacturer.endswith('S.L.\n'):
                        manufacturer = manufacturer[:-4]
                    if manufacturer.endswith('B.V.\n'):
                        manufacturer = manufacturer[:-4]
                    if manufacturer.endswith('N.V.\n'):
                        manufacturer = manufacturer[:-4]
                    if manufacturer.endswith('GmbH & Co KG\n'):
                        manufacturer = manufacturer[:-12]
                    elif manufacturer.startswith('NIBE'):
                        manufacturer = 'Nibe\n'
                    elif manufacturer.startswith('Nibe'):
                        manufacturer = 'Nibe\n'
                    elif manufacturer.startswith('Mitsubishi'):
                        manufacturer = 'Mitsubishi\n'
                    elif manufacturer.startswith('Ochsner'):
                        manufacturer = 'Ochsner\n'
                    elif manufacturer.startswith('OCHSNER'):
                        manufacturer = 'Ochsner\n'
                    elif manufacturer.startswith('Viessmann'):
                        manufacturer = 'Viessmann\n'

                elif (lines.endswith('Date\n') == 1):
                    if contents[i-2].startswith('Phase'):
                        continue
                    date = (contents[i])
                    if (date == 'basis\n'):
                        date = contents[i - 3]
                        date = date[14:]
                elif (lines.startswith('Model:') == 1):
                    modul = (contents[i - 2])
                    splindoor_low = 'NaN'
                    splindoor_medium = 'NaN'
                    sploutdoor_low = 'NaN'
                    sploutdoor_medium = 'NaN'
                elif lines.endswith('Type\n'):
                    heatpumpType = contents[i][:-1]
                    if heatpumpType.startswith('A'):
                        heatpumpType = 'Outdoor Air/Water'
                    if heatpumpType.startswith('Eau glycol'):
                        heatpumpType = 'Brine/Water'
                elif (lines.startswith('Sound power level indoor')):

                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            splindoor_low = contents[i + 4][:-7]
                            splindoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        splindoor_medium = contents[i + 4][:-7]
                        splindoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            splindoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            splindoor_low = contents[i][:-7]

                        if (contents[i - 6].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        else:
                            splindoor_low = contents[i][:-7]
                            splindoor_medium = contents[i][:-7]

                elif (lines.startswith('Sound power level outdoor')):
                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            sploutdoor_low = contents[i + 4][:-7]
                            sploutdoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        sploutdoor_medium = contents[i + 4][:-7]
                        sploutdoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            sploutdoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        else:
                            sploutdoor_low = contents[i][:-7]
                            sploutdoor_medium = contents[i][:-7]

                elif (lines.startswith('Puissance acoustique extérieure')):
                    b = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            sploutdoor_low = contents[i + 4][:-7]
                            sploutdoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        sploutdoor_medium = contents[i + 4][:-7]
                        sploutdoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            sploutdoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        else:
                            sploutdoor_low = contents[i][:-7]
                            sploutdoor_medium = contents[i][:-7]
                elif (lines.startswith('Potencia sonora de la unidad interior')):
                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            splindoor_low = contents[i + 4][:-7]
                            splindoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        splindoor_medium = contents[i + 4][:-7]
                        splindoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            splindoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        else:
                            splindoor_low = contents[i][:-7]
                            splindoor_medium = contents[i][:-7]
                elif (lines.startswith('Potencia sonora de la unidad exterior')):
                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            sploutdoor_low = contents[i + 4][:-7]
                            sploutdoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        sploutdoor_medium = contents[i + 4][:-7]
                        sploutdoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            sploutdoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        else:
                            sploutdoor_low = contents[i][:-7]
                            sploutdoor_medium = contents[i][:-7]
                elif (lines.startswith('Nivel de Potência sonora interior')):
                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            splindoor_low = contents[i + 4][:-7]
                            splindoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        splindoor_medium = contents[i + 4][:-7]
                        splindoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            splindoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        else:
                            splindoor_low = contents[i][:-7]
                            splindoor_medium = contents[i][:-7]
                elif (lines.startswith('Nivel de Potência sonora exterior')):
                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            sploutdoor_low = contents[i + 4][:-7]
                            sploutdoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        sploutdoor_medium = contents[i + 4][:-7]
                        sploutdoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            sploutdoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        else:
                            sploutdoor_low = contents[i][:-7]
                            sploutdoor_medium = contents[i][:-7]
                elif (lines.startswith('Livello di potenza acustica interna')):
                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            splindoor_low = contents[i + 4][:-7]
                            splindoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        splindoor_medium = contents[i + 4][:-7]
                        splindoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            splindoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            splindoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            splindoor_medium = contents[i + 2][:-7]
                        else:
                            splindoor_low = contents[i][:-7]
                            splindoor_medium = contents[i][:-7]
                elif (lines.startswith('Livello di potenza acustica externa')):
                    SPL = 1
                    if (contents[i].startswith('Low')):
                        if contents[i + 2].startswith('Medium'):
                            sploutdoor_low = contents[i + 4][:-7]
                            sploutdoor_medium = contents[i + 6][:-7]
                    if contents[i].startswith('Medium'):
                        sploutdoor_medium = contents[i + 4][:-7]
                        sploutdoor_low = contents[i + 6][:-7]
                    elif (contents[i].endswith('dB(A)\n')):
                        if (contents[i - 3].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 3].startswith('Medium')):
                            sploutdoor_medium = contents[i][:-7]
                        if (contents[i - 6].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 6].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        if (contents[i - 4].startswith('Low')):
                            sploutdoor_low = contents[i][:-7]
                        if (contents[i - 4].startswith('Medium')):
                            sploutdoor_medium = contents[i + 2][:-7]
                        else:
                            sploutdoor_low = contents[i][:-7]
                            sploutdoor_medium = contents[i][:-7]
                elif (lines == 'Refrigerant\n'):
                    if (contents[i - 3] == 'Mass Of\n'):
                        continue
                    refrigerant = (contents[i])
                elif (lines.startswith('Mass Of') == 1 or lines.startswith('Mass of')):
                    if (lines == 'Mass Of\n'):
                        mass = contents[i + 1]
                    elif (lines.endswith('kg\n') == 1):
                        mass = contents[i - 2]
                        mass = mass[20:]
                    else:
                        mass = contents[i]

                elif lines.startswith('Average'):
                    climate = 'average'
                elif lines.startswith('Cold'):
                    climate = 'cold'
                elif lines.startswith('Warmer Climate'):
                    climate = 'warm'

                elif (lines.startswith('EN') == 1):
                    if (p == 1):
                        Poff.append(poff)
                        Psb.append(psb)
                    if (p == 2):
                        Poff.append(poff)
                        Poff.append(poff)
                        Psb.append(psb)
                        Psb.append(psb_medium)
                    guideline = (contents[i - 2])
                    d = 0  # Medium or Low Content
                    p = 0  # -15 yes or no

                    NumberOfTestsPerNorm = []
                    if (contents[i - 1].startswith('Low') == 1):
                        d = 0
                        continue
                    if (contents[i - 1] == '\n'):
                        continue
                    if (contents[i - 1].startswith('Medium')):
                        d = 1
                    else:
                        d = 0
                if lines.startswith('Prated'):
                    prated_low = contents[i][:-4]
                    if (contents[i + 2].endswith('kW\n')):
                        prated_medium = contents[i + 2][:-4]


                elif (lines.startswith('Pdh Tj = -15°C') == 1):  # check
                    if (contents[i].startswith('Cdh') == 1):  # wrong content
                        continue
                    if (contents[i].startswith('EHPA') == 1):  # wrong content
                        print(contents[i+4:i+10])
                        continue
                    if (contents[i].endswith('Cdh\n') == 1):  # wrong content
                        continue
                    elif (contents[i] == '\n'):  # no content
                        continue
                    else:
                        minusfifteen_low = contents[i]
                        P_th.append(minusfifteen_low[:-4])
                        T_in.append('-15')
                        if d == 0:  # first low than medium Temperatur
                            if (climate == 'average'):
                                T_out.append('35')
                            elif (climate == 'cold'):
                                T_out.append('32')
                            elif (climate == 'warm'):
                                T_out.append('35')

                        if d == 1:  # first medium Temperature
                            if (climate == 'average'):
                                T_out.append('55')
                            elif (climate == 'cold'):
                                T_out.append('49')
                            elif (climate == 'warm'):
                                T_out.append('55')

                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        Mass.append(mass[:-4])
                        Prated.append(prated_low)
                        SPLindoor.append(splindoor_low)
                        # SPLindoor.append(splindoor_medium)
                        SPLoutdoor.append(sploutdoor_low)
                        # SPLoutdoor.append(sploutdoor_medium)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                        Type.append(heatpumpType)
                        if (contents[i + 2].startswith('COP')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('Disclaimer')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('EHPA')):  # End of page
                            if (len(contents)- i)<10:
                                continue
                            elif (contents[i + 8].startswith('COP')):  # end of page plus no medium heat
                                continue
                        minusfifteen_medium = contents[i + 2]

                        P_th.append(minusfifteen_medium[:-4])
                        T_in.append('-15')
                        if (climate == 'average'):
                            T_out.append('55')
                        elif (climate == 'cold'):
                            T_out.append('49')
                        elif (climate == 'warm'):
                            T_out.append('55')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        Mass.append(mass[:-4])
                        Prated.append(prated_medium)
                        # SPLindoor.append(splindoor_low)
                        SPLindoor.append(splindoor_medium)
                        # SPLoutdoor.append(sploutdoor_low)
                        SPLoutdoor.append(sploutdoor_medium)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                elif (lines.startswith('COP Tj = -15°C')):
                    if (contents[i] == '\n'):
                        continue
                    if (contents[i].startswith('EHPA')):
                        continue
                    COP.append(contents[i][:-1])
                    NumberOfTestsPerModule.append(i)
                    p = 1

                    if (contents[i + 2].startswith('Pdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('Cdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('EHPA')):  # no medium Climate
                        continue
                    COP.append(contents[i + 2][:-1])
                    NumberOfTestsPerModule.append(i)
                    p = 2


                elif (lines.startswith('Pdh Tj = -7°C') == 1):  # check
                    minusseven_low = contents[i]
                    P_th.append(minusseven_low[:-4])
                    T_in.append('-7')
                    if d == 0:  # first low than medium Temperatur
                        if (climate == 'average'):
                            T_out.append('34')
                        elif (climate == 'cold'):
                            T_out.append('30')
                        elif (climate == 'warm'):
                            T_out.append('35')

                    if d == 1:  # first medium Temperature
                        if (climate == 'average'):
                            T_out.append('52')
                        elif (climate == 'cold'):
                            T_out.append('44')
                        elif (climate == 'warm'):
                            T_out.append('55')

                    Modul.append(modul[7:-1])
                    Manufacturer.append(manufacturer[:-1])
                    Date.append(date[:-1])
                    Refrigerant.append(refrigerant[:-1])
                    Mass.append(mass[:-4])
                    Prated.append(prated_low)
                    SPLindoor.append(splindoor_low)
                    # SPLindoor.append(splindoor_medium)
                    SPLoutdoor.append(sploutdoor_low)
                    # SPLoutdoor.append(sploutdoor_medium)
                    Type.append(heatpumpType)
                    Guideline.append(guideline[:-1])
                    Climate.append(climate)

                    if (contents[i + 2].startswith('COP') == 1):
                        continue
                    else:
                        minusseven_medium = contents[i + 2]
                        P_th.append(minusseven_medium[:-4])
                        T_in.append('-7')
                        if (climate == 'average'):
                            T_out.append('52')
                        elif (climate == 'cold'):
                            T_out.append('44')
                        elif (climate == 'warm'):
                            T_out.append('55')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        # SPLindoor.append(splindoor_low)
                        SPLindoor.append(splindoor_medium)
                        # SPLoutdoor.append(sploutdoor_low)
                        SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_medium)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                elif (lines.startswith('COP Tj = -7°C')):
                    COP.append(contents[i][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)
                    if (contents[i + 2].startswith('Pdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('Cdh')):  # no medium Climate
                        continue
                    COP.append(contents[i + 2][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)


                elif (lines.startswith('Pdh Tj = +2°C') == 1):
                    if (contents[i].endswith('Cdh\n') == 1):  # wrong content
                        continue
                    if (contents[i] == '\n'):  # no content
                        continue
                    else:
                        plustwo_low = contents[i]
                        P_th.append(plustwo_low[:-4])
                        T_in.append('2')
                        if d == 0:  # first low than medium Temperatur
                            if (climate == 'average'):
                                T_out.append('30')
                            elif (climate == 'cold'):
                                T_out.append('27')
                            elif (climate == 'warm'):
                                T_out.append('35')

                        if d == 1:  # first medium Temperature
                            if (climate == 'average'):
                                T_out.append('42')
                            elif (climate == 'cold'):
                                T_out.append('37')
                            elif (climate == 'warm'):
                                T_out.append('55')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        SPLindoor.append(splindoor_low)
                        # SPLindoor.append(splindoor_medium)
                        SPLoutdoor.append(sploutdoor_low)
                        # SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_low)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                        if (contents[i + 2].startswith('COP')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('Disclaimer')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('EHPA')):  # End of page
                            if (contents[i + 8].startswith('COP')):  # end of page plus no medium heat
                                continue
                        plustwo_medium = contents[i + 2]
                        # if(plustwo_low[:-1].endswith('kW')==0):#test
                        # print(plustwo_low[:-1])
                        # if(plustwo_medium[:-1].endswith('kW')==0):#test
                        # print(file.name)#plustwo_medium[:-1]

                        P_th.append(plustwo_medium[:-4])
                        T_in.append('2')
                        if (climate == 'average'):
                            T_out.append('42')
                        elif (climate == 'cold'):
                            T_out.append('37')
                        elif (climate == 'warm'):
                            T_out.append('55')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        # SPLindoor.append(splindoor_low)
                        SPLindoor.append(splindoor_medium)
                        # SPLoutdoor.append(sploutdoor_low)
                        SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_medium)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                elif (lines.startswith('COP Tj = +2°C')):  # check
                    if (contents[i] == '\n'):  # no infos
                        continue
                    if (contents[i].startswith('EHPA')):  # end of page
                        print(file.name)
                        continue
                    if (contents[i + 2].startswith('Warmer')):  # usless infos
                        continue
                    if (contents[i] == 'n/a\n'):  # usless infos
                        continue
                    COP.append(contents[i][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)

                    if (contents[i + 2].startswith('Pdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('Cdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('EHPA')):  # no medium Climate
                        continue
                    COP.append(contents[i + 2][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)


                elif (lines.startswith('Pdh Tj = +7°C') == 1):
                    if (contents[i].endswith('Cdh\n') == 1):  # wrong content
                        continue
                    if (contents[i] == '\n'):  # no content
                        continue
                    else:
                        plusseven_low = contents[i]
                        P_th.append(plusseven_low[:-4])
                        T_in.append('7')
                        if d == 0:  # first low than medium Temperatur
                            if (climate == 'average'):
                                T_out.append('27')
                            elif (climate == 'cold'):
                                T_out.append('25')
                            elif (climate == 'warm'):
                                T_out.append('31')

                        if d == 1:  # first medium Temperature
                            if (climate == 'average'):
                                T_out.append('36')
                            elif (climate == 'cold'):
                                T_out.append('32')
                            elif (climate == 'warm'):
                                T_out.append('46')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        SPLindoor.append(splindoor_low)
                        # SPLindoor.append(splindoor_medium)
                        SPLoutdoor.append(sploutdoor_low)
                        # SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_low)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                        if (contents[i + 2].startswith('COP')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('Disclaimer')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('EHPA')):  # End of page
                            if (contents[i + 8].startswith('COP')):  # end of page plus no medium heat
                                continue
                        plusseven_medium = contents[i + 2]

                        P_th.append(plusseven_medium[:-4])
                        T_in.append('7')
                        if (climate == 'average'):
                            T_out.append('36')
                        elif (climate == 'cold'):
                            T_out.append('32')
                        elif (climate == 'warm'):
                            T_out.append('46')

                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        # SPLindoor.append(splindoor_low)
                        SPLindoor.append(splindoor_medium)
                        # SPLoutdoor.append(sploutdoor_low)
                        SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_medium)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                elif (lines.startswith('COP Tj = +7°C')):  # check
                    if (contents[i] == '\n'):  # no infos
                        continue
                    if (contents[i].startswith('EHPA')):  # end of page
                        continue
                    if (contents[i + 2].startswith('Warmer')):  # usless infos
                        continue
                    if (contents[i] == 'n/a\n'):  # usless infos
                        continue
                    COP.append(contents[i][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)

                    if (contents[i + 2].startswith('Pdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('Cdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('EHPA')):  # no medium Climate
                        continue
                    COP.append(contents[i + 2][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)


                elif (lines.startswith('Pdh Tj = 12°C') == 1):

                    if (contents[i].endswith('Cdh\n') == 1):  # wrong content
                        continue
                    if (contents[i] == '\n'):  # no content
                        continue
                    if (contents[i].startswith('EHPA Secretariat') == 1):
                        plustwelfe_low = (contents[i - 11])

                        P_th.append(plustwelfe_low[:-4])
                        T_in.append('12')
                        if (climate == 'average'):
                            T_out.append('24')
                        elif (climate == 'cold'):
                            T_out.append('24')
                        elif (climate == 'warm'):
                            T_out.append('26')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        SPLindoor.append(splindoor_low)
                        # SPLindoor.append(splindoor_medium)
                        SPLoutdoor.append(sploutdoor_low)
                        # SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_low)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                        plustwelfe_medium = (contents[i - 9])

                        P_th.append(plustwelfe_medium[:-4])
                        T_in.append('12')
                        if (climate == 'average'):
                            T_out.append('30')
                        elif (climate == 'cold'):
                            T_out.append('28')
                        elif (climate == 'warm'):
                            T_out.append('34')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        # SPLindoor.append(splindoor_low)
                        SPLindoor.append(splindoor_medium)
                        # SPLoutdoor.append(sploutdoor_low)
                        SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_medium)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                    else:
                        plustwelfe_low = contents[i]

                        P_th.append(plustwelfe_low[:-4])
                        T_in.append('12')
                        if d == 0:  # first low than medium Temperatur
                            if (climate == 'average'):
                                T_out.append('24')
                            elif (climate == 'cold'):
                                T_out.append('24')
                            elif (climate == 'warm'):
                                T_out.append('26')

                        if d == 1:  # first medium Temperature
                            if (climate == 'average'):
                                T_out.append('30')
                            elif (climate == 'cold'):
                                T_out.append('28')
                            elif (climate == 'warm'):
                                T_out.append('34')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        SPLindoor.append(splindoor_low)

                        SPLoutdoor.append(sploutdoor_low)

                        Mass.append(mass[:-4])
                        Prated.append(prated_low)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                        if (contents[i + 2].startswith('COP')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('Disclaimer')):  # for PDF without medium heat
                            continue
                        if (contents[i + 2].startswith('EHPA')):  # End of page
                            if (contents[i + 8].startswith('COP')):  # end of page plus no medium heat
                                continue

                        plustwelfe_medium = contents[i + 2]
                        P_th.append(plustwelfe_medium[:-4])
                        T_in.append('12')
                        if (climate == 'average'):
                            T_out.append('30')
                        elif (climate == 'cold'):
                            T_out.append('28')
                        elif (climate == 'warm'):
                            T_out.append('34')
                        Modul.append(modul[7:-1])
                        Manufacturer.append(manufacturer[:-1])
                        Date.append(date[:-1])
                        Refrigerant.append(refrigerant[:-1])
                        # SPLindoor.append(splindoor_low)
                        SPLindoor.append(splindoor_medium)

                        SPLoutdoor.append(sploutdoor_medium)
                        Mass.append(mass[:-4])
                        Prated.append(prated_medium)
                        Type.append(heatpumpType)
                        Guideline.append(guideline[:-1])
                        Climate.append(climate)

                elif (lines.startswith('COP Tj = 12°C')):  # check
                    if (contents[i] == '\n'):  # no infos
                        continue
                    if (contents[i].startswith('EHPA')):  # end of page
                        print('W')
                        continue
                    if (contents[i + 2].startswith('Warmer')):  # usless infos
                        continue
                    if (contents[i] == 'n/a\n'):  # usless infos
                        continue
                    COP.append(contents[i][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)

                    if (contents[i + 2].startswith('Pdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('Cdh')):  # no medium Climate
                        continue
                    if (contents[i + 2].startswith('EHPA')):  # no medium Climate
                        continue
                    COP.append(contents[i + 2][:-1])
                    NumberOfTestsPerNorm.append(i)
                    NumberOfTestsPerModule.append(i)


                elif (lines.startswith('Poff')):
                    l = 0  # l shows if Poff Medium is different to Poff Low Temperature
                    c = 2  # c is just an iterator to print every second Poff
                    poff = contents[i][:-2]
                    if poff.endswith(' '):
                        poff = poff[:-1]
                        if poff.endswith('.00'):
                            poff = poff[:-3]
                    second_poff = contents[i + 2][:-2]
                    if second_poff.endswith(' '):
                        second_poff = second_poff[:-1]
                        if second_poff.endswith('.00'):
                            second_poff = second_poff[:-3]
                    if (poff != second_poff):  # see if Poff Medium to Poff low
                        if (contents[i + 2].endswith('W\n')):
                            if (contents[i + 2] != 'W\n'):
                                l = 1
                    for Tests in NumberOfTestsPerNorm:
                        if l == 0:
                            Poff.append(poff)
                        if l == 1:
                            c += 1
                            if c % 2 == 1:
                                Poff.append(poff)
                            if c % 2 == 0:
                                Poff.append(second_poff)
                elif (lines.startswith('PSB')):
                    l = 0  # l shows if Poff Medium is different to Poff Low Temperature
                    c = 2  # c is just an iterator to print every second Poff
                    psb = contents[i][:-2]
                    if psb.endswith(' '):
                        psb = psb[:-1]
                        if psb.endswith('.00'):
                            psb = psb[:-3]
                    psb_medium = contents[i + 2][:-2]
                    if psb_medium.endswith(' '):
                        psb_medium = psb_medium[:-1]
                        if psb_medium.endswith('.00'):
                            psb_medium = psb_medium[:-3]
                    if (psb != psb_medium):  # see if Poff Medium to Poff low
                        if (contents[i + 2].endswith('W\n')):
                            if (contents[i + 2] != 'W\n'):
                                l = 1

                    for Tests in NumberOfTestsPerNorm:
                        if l == 0:
                            Psb.append(psb)
                        if l == 1:
                            c += 1
                            if c % 2 == 1:
                                Psb.append(psb)
                            if c % 2 == 0:
                                Psb.append(psb_medium)

            if p == 1:
                Poff.append(poff)
                Psb.append(psb)
            if p == 2:
                Poff.append(poff)
                Poff.append(second_poff)
                Psb.append(psb)
                Psb.append(psb_medium)

['\x0cPage 12 of 16\n', 'This information was generated by the HP KEYMARK database on 18 Mar 2022\n']
['\x0cPage 50 of 85\n', 'This information was generated by the HP KEYMARK database on 29 Mar 2022\n']
['\x0cPage 57 of 85\n', 'This information was generated by the HP KEYMARK database on 29 Mar 2022\n']
['\x0cPage 64 of 85\n', 'This information was generated by the HP KEYMARK database on 29 Mar 2022\n']
['\x0cPage 71 of 85\n', 'This information was generated by the HP KEYMARK database on 29 Mar 2022\n']
['\x0cPage 29 of 43\n', 'This information was generated by the HP KEYMARK database on 29 Mar 2022\n']
['\x0cPage 36 of 43\n', 'This information was generated by the HP KEYMARK database on 29 Mar 2022\n']
['\x0cPage 12 of 16\n', 'This information was generated by the HP KEYMARK database on 18 Mar 2022\n']
['\x0cPage 29 of 43\n', 'This information was generated by the HP KEYMARK database on 18 Mar 2022\n']
['\x0cPage 36 of 43\n', 'This information was generated by the HP KEYMARK database

In [3]:
df['Manufacturer'] = Manufacturer
df['Model'] = Modul
df['Date'] = Date
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
df['Type'] = Type
df['SPL indoor [dBA]'] = SPLindoor
df['SPL outdoor [dBA]'] = SPLoutdoor
df['Refrigerant'] = Refrigerant
df['Mass of Refrigerant [kg]'] = Mass
#df['Poff [W]'] = Poff
#df['Poff [W]'] = df['Poff [W]'].astype(int)
#df['PSB [W]'] = Psb
#df['PSB [W]'] = df['PSB [W]'].astype(int)
df['Prated [W]'] = Prated

df['Guideline'] = Guideline
df['Climate'] = Climate
df['T_in [°C]'] = T_in
df['T_in [°C]'] = df['T_in [°C]'].astype(int)
df['T_out [°C]'] = T_out
df['T_out [°C]'] = df['T_out [°C]'].astype(int)
"""  
T_out for Low Temperature
        T-in:   -15 -7  2   7   12

Cold Climate    32  30  27  25  24
Average Climate 35  34  30  27  24
Warm Climate    35  35  35  31  26    


T_out for Medium Temperature
        T-in:   -15 -7  2   7   12

Cold Climate    49  44  37  32  28
Average Climate 55  52  42  36  30
Warm Climate    55  55  55  46  34                    
"""
df['P_th [W]'] = P_th
df['P_th [W]'] = ((df['P_th [W]'].astype(float)) * 1000).astype(int)
df['COP'] = COP
df['COP'] = round(df['COP'].astype(float), 2)
df['P_el [W]'] = round(df['P_th [W]'] / df['COP'])
df['P_el [W]'] = df['P_el [W]'].fillna(0).astype(int)
df['PSB [W]'] = df['PSB [W]'].where(df['PSB [W]'] > df['Poff [W]'],
                                df['Poff [W]'])  # Poff should not be bigger than PSB
df.drop(columns=['Poff [W]'], inplace=True)  # not needed anymore
filt = df['P_th [W]'] < 50  # P_th too small
df.drop(index=df[filt].index, inplace=True)
# add T_amb and change T_in to right values
df['T_amb [°C]'] = df['T_in [°C]']
filt = df['Type'] == 'Brine/Water'
df.loc[filt, 'T_in [°C]'] = 0
filt = df['Type'] == 'Water/Water'
df.loc[filt, 'T_in [°C]'] = 10
df = df[
['Manufacturer', 'Model', 'Date', 'Type', 'Refrigerant', 'Mass of Refrigerant [kg]', 'PSB [W]', 'Prated [W]',
        'SPL indoor [dBA]', 'SPL outdoor [dBA]', 'Climate', 'T_amb [°C]', 'T_in [°C]', 'T_out [°C]', 'P_th [W]',
        'P_el [W]', 'COP']]
df.sort_values(by=['Manufacturer', 'Model'], inplace=True)
os.chdir("../")
df.to_csv(r'../output/database_heating.csv', index=False)
os.chdir('../hplib/')

ValueError: could not convert string to float: '13.18 kW 12.23'

In [ ]:
df

,Manufacturer,Model,Date,Type,SPL indoor [dBA],SPL outdoor [dBA],Refrigerant,Mass of Refrigerant [kg],Prated [W],Guideline,Climate,T_in [°C],T_out [°C],P_th [W]
0,Clivet,WSAN-YSi 30.2,2022-02-21,Outdoor Air/Water,NaN,82,R32,17.5,51.00,EN 14825,average,-7,34,45.02
1,Clivet,WSAN-YSi 30.2,2022-02-21,Outdoor Air/Water,NaN,82,R32,17.5,51.00,EN 14825,average,2,30,28.17
2,Clivet,WSAN-YSi 30.2,2022-02-21,Outdoor Air/Water,NaN,82,R32,17.5,51.00,EN 14825,average,7,27,26.16
3,Clivet,WSAN-YSi 30.2,2022-02-21,Outdoor Air/Water,NaN,82,R32,17.5,51.00,EN 14825,average,12,24,31.74
4,Clivet,WSAN-YSi 35.2,2022-02-21,Outdoor Air/Water,NaN,83,R32,17.5,55.00,EN 14825,average,-7,34,48.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65477,Groupe Atlantic,Austria Email LWPK 6 A.I.,2020-03-04,Outdoor Air/Water,40,57,R32,0.97,5.00,EN 14825,average,2,42,2.90
65478,Groupe Atlantic,Austria Email LWPK 6 A.I.,2020-03-04,Outdoor Air/Water,40,57,R32,0.97,6.00,EN 14825,average,7,27,2.10
65479,Groupe Atlantic,Austria Email LWPK 6 A.I.,2020-03-04,Outdoor Air/Water,40,57,R32,0.97,5.00,EN 14825,average,7,36,1.80
65480,Groupe Atlantic,Austria Email LWPK 6 A.I.,2020-03-04,Outdoor Air/Water,40,57,R32,0.97,6.00,EN 14825,average,12,24,2.40


In [5]:
i=0
for date in Date:
    if date.startswith('3x'):
        print(model,i)
    i+=1

In [ ]:
%%html
<style>
table {float:left}
</style>

## **1. Definitions** <a class="anchor" id="definitions" name="definitions"></a>

**Abbreviations**

| Abbreviation | Meaning |
| :--- | :--- |
| P_th | Thermal output power in W |
| P_el | Electical input Power in W |
| COP | Coefficient of performance |
| EER | Energy Efficiency Ratio |
| T_in | Input temperature in °C at primary side of the heat pump |
| T_out | Output temperature in °C at secondary side of the heat pump |
| T_amb | Ambient temperature in °C |
| P_th_h_ref | Thermal heating output power in W at T_in = -7 °C and T_out = 52 °C |
| P_el_h_ref | Elecrical input power (heating) in W at T_in = -7 °C and T_out = 52 °C |
| COP_ref | Coefficient of performance at T_in = -7 °C and T_out = 52 °C |
| P_th_c_ref | Thermal cooling output power in W at T_in = 35 °C and T_out = 7 °C |
| P_el_c_ref | Elecrical input power (cooling) in W at T_in = 35 °C and T_out = 7 °C |
| p1-p4 | Fit-Parameters for Fit-Function |



**Group IDs**

| Group ID | Type | Subtype |
| :--- | :--- | :--- |
| 1 | Outdoor Air / Water | Regulated | 
| 2 | Brine / Water | Regulated | 
| 3 | Water / Water | Regulated | 
| 4 | Outdoor Air / Water | On-Off | 
| 5 | Brine / Water | On-Off | 
| 6 | Water / Water | On-Off |

## **2. Database preparation** <a class="anchor" id="database-preparation" name="database-preparation"></a>

This section is only for documentation regarding the development of the final database. It's not neccesary to run this code again.

1. we downloaded all manufacturer data from https://keymark.eu/en/products/heatpumps/certified-products .
2. then we unzipped the files to the `input` folder and used the bash-skript `pdf2txt.sh` to convert pdf into txt.
3. afterwards we used the following functions to create and extent the heatpump keymark database.

In [ ]:
# Import keymark data and save to csv database
db.import_heating_data('txt_new')
# -> this creates /output/database_heating.csv
db.import_cooling_data('txt_new')
# -> this creates /output/database_cooling.csv

In [ ]:
# Reduce to climate measurement series with average climate, delete redundant entries and save to csv sub-database
db.reduce_heating_data('database_heating.csv','average')
# -> this creates /output/database_heating_average.csv

**Process heating database**

In [ ]:
# Normalize electrical and thermal power from the keymark database to values from setpoint T_in = -7 °C and T_out = 52 °C
db.normalize_heating_data('database_heating_average.csv')
# -> this creates /output/database_heating_average_normalized.csv

In [ ]:
# Identify subtypes like on-off or regulated heat pumps and assign group id depending on its type and subtype
db.identify_subtypes('database_heating_average_normalized.csv')
# -> this creates /output/database_heating_average_normalized_subtypes.csv

In [ ]:
# Calculate parameters p1-p4 for P_th, P_el and COP with a least-square fit approach
# based on  K. Schwamberger: „Modellbildung und Regelung von Gebäudeheizungsanlagen 
# mit Wärmepumpen“, VDI Verlag, Düsseldorf, Fortschrittsberichte VDI Reihe 6 Nr. 263, 1991.
db.calculate_heating_parameters('database_heating_average_normalized_subtypes.csv')
# -> this creates /output/hplib_database_heating.csv
# -> this creates /hplib/hplib_database.csv

In [ ]:
# Many heat pump models have redundant entries because of different controller or storage configurations.
# Reduce to unique heat pump models.
db.reduce_to_unique()
# -> this overwrites the /output/hplib_database_heating.csv

In [ ]:
# Calculate the relative error (RE) for each set point of every heat pump
db.validation_relative_error_heating()
# -> this creates /output/database_heating_average_normalized_subtypes_validation.csv

In [ ]:
# Calculate the mean absolute percentage error (MAPE) for every heat pump
db.validation_mape_heating()
# -> this overwrites the /output/hplib_database_heating.csv

**Process cooling database**

Overall there are not so many unique Keymark heat pumps for cooling (34 models) in comparison to heating (505 models).

Out of the 34 models only 4 heat pumps had set points at different outflow temperature. 
With our fit method it is not possible to fit only over one outflow temperature. For that reason we added another
set point at 18°C output temperature based on the heat pumps we had with this condition in Keymark. For that purpose, we identified multiplication factors for eletrical power and eer between 7°C and 18°C secondary output temperature. The mean value of that are used to calculate the electrical power and EER at 18°C for other heat pumps:

```
P_el at 18°C = P_el at 7°C * multiplication factor 
EER at 18°C = EER at 7°C * multiplication factor
```

| Outside Tempertature | Multiplication factor for P_el | Multiplication factor for EER |
| :--- | :--- | :--- |
| 35 | 0.85 | 1.21 |
| 30 | 0.82 | 1.21 |
| 25 | 0.77 | 1.20 |
| 20 | 0.63 | 0.95 |


In [ ]:
#Only use heatpumps which are unique and also in the heating library
db.reduce_cooling_data()
#this generates /output/database_cooling_reduced.csv

In [ ]:
# Normalize electrical and thermal power from the keymark database to values from setpoint T_outside = 35 °C and T_out = 7 °C
db.normalize_and_add_cooling_data()
#-> this creates /output/database_cooling_reduced_normalized.csv

In [ ]:
# Used the same fit method like in heating
# except: for P_el the point 20/7 is ignored
db.calculate_cooling_parameters()
# -> this overwrites /hplib/hplib_database.csv

In [ ]:
# Calculate the relative error (RE) for each set point of every heat pump
db.validation_relative_error_cooling()
# this creates /output/database_cooling_reduced_normalized_subtypes_validation.csv

In [ ]:
# Calculate the mean absolute percentage error (MAPE) for every heat pump
db.validation_mape_cooling()
# -> this overwrites the /hplib/hplib_database.csv

**Create generic heat pump models**

In [ ]:
# Calculate generic heat pump models for each group id
# for cooling: there is only a generic heat pump of type air/water | regulated available
db.add_generic()
# -> this overwrites the /hplib/hplib_database.csv

**Hint:** The csv files in the `output` folder are for documentation and validation purpose. The code `hplib.py` and database `hplib_database` files, which are meant to be used for simulations, are located in the `hplib` folder. 

## **3. Work with database** <a class="anchor" id="work-with-database" name="work-with-database"></a>

### **3.1 Load database**  <a class="anchor" id="load-database" name="load-database"></a>

Simply execute the command without arguments and you will get a DataFrame with the complete list of manufacturers and models. Now you are able to view, filter or sort the database.

In [ ]:
database = hpl.load_database()
database

### **3.2 Load specific model**  <a class="anchor" id="load-specific-model" name="load-specific-model"></a>

To get the parameters of a specific heat pump model, use the `get_parameters()` method with a specific Model name from the database.
You will get a DataFrame with all parameters including the mean absolute percentage errors (MAPE) for this model.

In [ ]:
parameters = hpl.get_parameters('i-SHWAK V4 06')
parameters

In [ ]:
hpl.get_parameters

### **3.3 Load generic model**  <a class="anchor" id="load-generic-model" name="load-generic-model"></a>

To get the parameters of a generic heat pump model, use the `get_parameters()` method with the following keyword arguments of a free choosen set point

* model='Generic'
* group_id: 1,2,4,5,6
* t_in: choose primary input temperature in °C
* t_out: choose secondary output temperature in °C
* p_th: choose thermal output power in W

You will get a DataFrame with all parameters for this generic model. For every group id the parameter set is based on the average parameters of all heat pumps of its group with an MAPE of less than 25%.

In [ ]:
parameters = hpl.get_parameters(model='Generic', group_id=1, t_in=-7, t_out=52, p_th=10000)
parameters

### **3.4 Get models with same fitting parameters**  <a class="anchor" id="load-generic-model" name="load-generic-model"></a>

To get a list of all models with same fitting parameters, use the `same_built_type()` method with the modelname from the hplib_datase.csv


In [ ]:
hpl.same_built_type('AW-YHPS08-H91 + AW-WHPS0810-N91')

## **4. How to simulate** <a class="anchor" id="how-to-simulate" name="how-to-simulate"></a>

With the Fit-Parameters p1-p4 for P_th, P_el and COP it is possible to calculate the results with the following methods:

1. P_th and P_el with Fit-Functions and `COP = P_th / P_el` or
2. P_th and COP with Fit-Functions and `P_el = P_th / COP` or
3. P_el and COP with Fut-Functions and `P_th = P_el * COP`

While the model by Schwarmberger [1] uses the first method, our validation showed, that the third method leads to better results. Therefore we decided to implement this in the `simulate` definition.

### **4.1 Simulate one timestep**  <a class="anchor" id="simulate-one-timestep" name="simulate-one-timestep"></a>

Please define a primary input temperature (t_in_primary), secondary input temperature (t_in_secondary),
ambient / outdoor temperature (t_amb) in °C and the parameters from the previous step. The t_in_secondary
is supposed to be heated up by 5 K which then results in output temperature.

Important:
* for air / water heat pumps t_in_primary and t_amb are supposed to be the same values.
* for brine / water or water / water heat pumps t_in_primary and t_amb are independent values.

In [ ]:
# Load parameters
parameters = hpl.get_parameters(model='Generic', group_id=1, t_in=-7, t_out=52, p_th=10000)
# Create heat pump object with parameters
heatpump = hpl.HeatPump(parameters)
# Simulate with values
# whereas mode = 1 is for heating and mode = 2 is for cooling
results = heatpump.simulate(t_in_primary=-7, t_in_secondary=47, t_amb=-7, mode=1)
print(pd.DataFrame([results]))

### **4.2 Simulate a timeseries**  <a class="anchor" id="simulate-a-timeseries" name="simulate-a-timeseries"></a>

The simulation approach could be to do a `for loop` around the method of simulating one timestep. But it will be faster to use the previous method with arrays as inputs. For usage you are allowed to define pandas.Series or arrays as input values. It is also possible to combine single values and pandas.Series / arrays.

*The next example uses a measured timeseries for ambient / outdoor temperature and secondary 
input temperature of a real heating system to demonstrate the simulation approach. The data represents
on year and has a temporal resolution of 1 minute*

In [ ]:
# Read input file with temperatures
df = pd.read_csv('../input/TestYear.csv')
df['T_amb'] = df['T_in_primary'] # air/water heat pump -> T_amb = T_in_primary
# Simulate with values
# Load parameters
parameters = hpl.get_parameters(model='Generic', group_id=1, t_in=-7, t_out=52, p_th=10000)
# Create heat pump object with parameters
heatpump = hpl.HeatPump(parameters)
# whereas mode = 1 is for heating and mode = 2 is for cooling
results = heatpump.simulate(t_in_primary=df['T_in_primary'].values, t_in_secondary=df['T_in_secondary'].values, t_amb=df['T_amb'].values, mode=1)
# Plot / Print some results
# example for distribution of COPs
results=pd.DataFrame.from_dict(results)
results['COP'].plot.hist(bins=50, title='Distribution of COP values') 
# Calclulate seasonal performance factor (SPF)
SPF = results['P_th'].mean() / results['P_el'].mean() 
print('The seasonal performance factor (SPF) for one year is = '+str(round(SPF,1)))

## **5. Example heat pump** <a class="anchor" id="example-heat-pump" name="example-heat-pump"></a>

To get a overview over the different operation conditions, this section plots the electrical and thermal power as well as the COP for all possible primary and secondary input temperatures
 
HEATING: **Schematic plot** of COP, P_el and P_th for an **generic air/water** heat pump: subtype = **regulated** 

In [ ]:
# Define Temperatures
T_in_primary = range(-20,31)
T_in_secondary = range(20,56)
T_in = numpy.array([])
T_out = numpy.array([])

# Load parameters of generic air/water | regulated
parameters = hpl.get_parameters('F2120-16', group_id=1, t_in=-7, t_out=35, p_th = 12280)
heatpump=hpl.HeatPump(parameters)
results=pd.DataFrame()
# Create input series
for t1 in T_in_primary:
    for t2 in T_in_secondary:
        T_in=numpy.append(T_in,t1)
        T_out=numpy.append(T_out,t2)
results=heatpump.simulate(t_in_primary=T_in, t_in_secondary=T_out, t_amb=T_in, mode=1)
results=pd.DataFrame.from_dict(results)

# Plot COP
fig1, ax1 = plt.subplots()
plot = plt.tricontourf(results['T_in'], results['T_out'], results['COP'])
ax1.tricontour(results['T_in'], results['T_out'], results['COP'], colors='k')
fig1.colorbar(plot)
ax1.set_title('COP')
ax1.set_xlabel('Primary input temperature [°C]')
ax1.set_ylabel('Secondary output temperature [°C]')
fig1.show

# Plot electrical input power
fig1, ax1 = plt.subplots()
plot = plt.tricontourf(results['T_in'], results['T_out'], results['P_el'])
ax1.tricontour(results['T_in'], results['T_out'], results['P_el'], colors='k')
fig1.colorbar(plot)
ax1.set_title('Eletrical input power [W]')
ax1.set_xlabel('Primary input temperature [°C]')
ax1.set_ylabel('Secondary output temperature [°C]')
fig1.show

# Plot thermal output power
fig1, ax1 = plt.subplots()
plot = plt.tricontourf(results['T_in'], results['T_out'], results['P_th'])
ax1.tricontour(results['T_in'], results['T_out'], results['P_th'], colors='k')
fig1.colorbar(plot)
ax1.set_title('Thermal output power [W]')
ax1.set_xlabel('Primary input temperature [°C]')
ax1.set_ylabel('Secondary output temperature [°C]')
fig1.show

COOLING: **Schematic plot** of EER, P_el and P_th for an **generic air/water** heat pump: subtype = **regulated** 

In [ ]:
# Define Temperatures
T_in_primary = range(20,36)
T_in_secondary = range(10,30)
T_in = numpy.array([])
T_out = numpy.array([])

# Load parameters of generic air/water | regulated
parameters = hpl.get_parameters('Generic', group_id=1, t_in=-7, t_out=52, p_th = 10000)
heatpump=hpl.HeatPump(parameters)
results=pd.DataFrame()
# Create input series
for t1 in T_in_primary:
    for t2 in T_in_secondary:
        T_in=numpy.append(T_in,t1)
        T_out=numpy.append(T_out,t2)
results=heatpump.simulate(t_in_primary=T_in, t_in_secondary=T_out, t_amb=T_in, mode=2)
results=pd.DataFrame.from_dict(results)
# Plot EER
fig1, ax1 = plt.subplots()
plot = plt.tricontourf(results['T_in'], results['T_out'], results['EER'])
ax1.tricontour(results['T_in'], results['T_out'], results['EER'], colors='k')
fig1.colorbar(plot)
ax1.set_title('EER')
ax1.set_xlabel('Primary input temperature [°C]')
ax1.set_ylabel('Secondary output temperature [°C]')
fig1.show

# Plot electrical input power
fig1, ax1 = plt.subplots()
plot = plt.tricontourf(results['T_in'], results['T_out'], results['P_el'])
ax1.tricontour(results['T_in'], results['T_out'], results['P_el'], colors='k')
fig1.colorbar(plot)
ax1.set_title('Eletrical input power [W]')
ax1.set_xlabel('Primary input temperature [°C]')
ax1.set_ylabel('Secondary output temperature [°C]')
fig1.show

# Plot thermal output power
fig1, ax1 = plt.subplots()
plot = plt.tricontourf(results['T_in'], results['T_out'], results['P_th'])
ax1.tricontour(results['T_in'], results['T_out'], results['P_th'], colors='k')
fig1.colorbar(plot)
ax1.set_title('Thermal output power [W]')
ax1.set_xlabel('Primary input temperature [°C]')
ax1.set_ylabel('Secondary output temperature [°C]')
fig1.show

## **6. Validation** <a class="anchor" id="validation" name="validation"></a>

The following plots will give you a detailed view on the differences between simulation and measurement from heat pump keymark. Therefore, all set points for all heat pumps are loaded from the file `database_heating_average_normalized_subtypes_validation.csv`.

In [ ]:
df = pd.read_csv('../output/database_heating_average_normalized_subtypes_validation.csv')

### **6.1 Air/Water | on/off** <a class="anchor" id="air-water-onoff" name="air-water-onoff"></a>

* The mean absolute percentage error (MAPE) over all heat pumps is
    * 5.4 % for COP
    * 2.5 % for P_el
    * 5.8 % for P_th
* The errors come from deviation mostly at low ambient / outside temperatures

In [ ]:
# Plot relative error for all heat pumps of type air/water | on-off
Group = 4
data = df.loc[(df['Group']==Group)]
data = data[['T_amb [°C]','RE_COP', 'RE_P_el', 'RE_P_th']]
ax = data.boxplot(by='T_amb [°C]', layout=(1,3), figsize=(10,5), showfliers=False)
ax[0].set_ylim(-60,60)
ax[0].set_ylabel('relative error in %')
data.abs().mean()[1:4] # mean absolute percentage error (MAPE)

In [ ]:
# Plot absolute values all heat pumps of type air/water | on-off as scatter plot
Group = 4
fig, ax = plt.subplots(1,3)
data = df.loc[(df['Group']==Group)]
data.plot.scatter(ax=ax[0], x='COP',y='COP_sim', alpha=0.3, figsize=(12,5), grid=True, title='Coefficient of performance')
data.plot.scatter(ax=ax[1], x='P_el [W]',y='P_el_sim', alpha=0.3, figsize=(15,5), grid=True, title='Electrical input power [W]')
data.plot.scatter(ax=ax[2], x='P_th [W]',y='P_th_sim', alpha=0.3, figsize=(15,5), grid=True, title='Thermal output power [W]')
ax[0].set_xlim(0,10)
ax[0].set_ylim(0,10)
ax[1].set_xlim(0,20000)
ax[1].set_ylim(0,20000)
ax[2].set_xlim(0,50000)
ax[2].set_ylim(0,50000)

### **6.2 Brine/Water | on/off** <a class="anchor" id="brine-water-onoff" name="brine-water-onoff"></a>

* The mean absolute percentage error (MAPE) over all heat pumps is
    * 3.9 % for COP
    * 1.7 % for P_el
    * 2.7 % for P_th
* **Important**: Validation data is only available at primary input temperature of 0 °C.  

In [ ]:
# Plot relative error for all heat pumps of type brine/water | on-off
Group = 5
data = df.loc[(df['Group']==Group)]
data = data[['T_amb [°C]','RE_COP', 'RE_P_el', 'RE_P_th']]
ax = data.boxplot(by='T_amb [°C]', layout=(1,3), figsize=(10,5), showfliers=False)
ax[0].set_ylim(-60,60)
ax[0].set_ylabel('relative error in %')
data.abs().mean()[1:4] # mean absolute percentage error (MAPE)

In [ ]:
# Plot absolute values all heat pumps of type brine/water | on-off as scatter plot
Group = 5
fig, ax = plt.subplots(1,3)
data = df.loc[(df['Group']==Group)]
data.plot.scatter(ax=ax[0], x='COP',y='COP_sim', alpha=0.3, figsize=(12,5), grid=True, title='Coefficient of performance')
data.plot.scatter(ax=ax[1], x='P_el [W]',y='P_el_sim', alpha=0.3, figsize=(15,5), grid=True, title='Electrical input power [W]')
data.plot.scatter(ax=ax[2], x='P_th [W]',y='P_th_sim', alpha=0.3, figsize=(15,5), grid=True, title='Thermal output power [W]')
ax[0].set_xlim(0,10)
ax[0].set_ylim(0,10)
ax[1].set_xlim(0,20000)
ax[1].set_ylim(0,20000)
ax[2].set_xlim(0,50000)
ax[2].set_ylim(0,50000)

### **6.3 Water/Water | on/off** <a class="anchor" id="water-water-onoff" name="water-water-onoff"></a>

* The mean absolute percentage error (MAPE) over all heat pumps is
    * 1.6 % for COP
    * 1.6 % for P_el
    * 2.4 % for P_th
* **Important**: Validation data is only available at primary input temperature of 10 °C.

In [ ]:
# Plot relative error for all heat pumps of type water/water | on-off
Group = 6
data = df.loc[(df['Group']==Group)]
data = data[['T_amb [°C]','RE_COP', 'RE_P_el', 'RE_P_th']]
ax = data.boxplot(by='T_amb [°C]', layout=(1,3), figsize=(10,5), showfliers=False)
ax[0].set_ylim(-60,60)
ax[0].set_ylabel('relative error in %')
data.abs().mean()[1:4] # mean absolute percentage error (MAPE)

In [ ]:
# Plot absolute values all heat pumps of type water/water | on-off as scatter plot
Group = 6
fig, ax = plt.subplots(1,3)
data = df.loc[(df['Group']==Group)]
data.plot.scatter(ax=ax[0], x='COP',y='COP_sim', alpha=0.3, figsize=(12,5), grid=True, title='Coefficient of performance')
data.plot.scatter(ax=ax[1], x='P_el [W]',y='P_el_sim', alpha=0.3, figsize=(15,5), grid=True, title='Electrical input power [W]')
data.plot.scatter(ax=ax[2], x='P_th [W]',y='P_th_sim', alpha=0.3, figsize=(15,5), grid=True, title='Thermal output power [W]')
ax[0].set_xlim(0,10)
ax[0].set_ylim(0,10)
ax[1].set_xlim(0,20000)
ax[1].set_ylim(0,20000)
ax[2].set_xlim(0,50000)
ax[2].set_ylim(0,50000)

### **6.4 Air/Water | regulated** <a class="anchor" id="air-water-regulated" name="air-water-regulated"></a>

**Heating**
* The mean absolute percentage error (MAPE) over all heat pumps is
    * 12.1 % for COP
    * 19.5 % for P_el
    * 23.5 % for P_th

**Cooling**
* The mean absolute percentage error (MAPE) over all heat pumps is
    * 4.8 % for EER
    * 16.5 % for P_el
    * 17.0 % for P_th
    
Because of different control strategies, the deviation over different heat pump models is much higher compared to on/off types.

#### **6.4.1 Heating** <a class="anchor" id="air-water-regulated-heating" name="air-water-regulated-heating"></a>

In [ ]:
# Plot relative error for all heat pumps of type air/water | regulated
Group = 1
data = df.loc[(df['Group']==Group)]
data = data[['T_amb [°C]','RE_COP', 'RE_P_el', 'RE_P_th']]
ax = data.boxplot(by='T_amb [°C]', layout=(1,3), figsize=(10,5), showfliers=False)
ax[0].set_ylim(-60,60)
ax[0].set_ylabel('relative error in %')
data.abs().mean()[1:4] # mean absolute percentage error (MAPE)

In [ ]:
# Plot absolute values all heat pumps of type air/water | regulated as scatter plot
Group = 1
fig, ax = plt.subplots(1,3)
data = df.loc[(df['Group']==Group)]
data.plot.scatter(ax=ax[0], x='COP',y='COP_sim', alpha=0.3, figsize=(12,5), grid=True, title='Coefficient of performance')
data.plot.scatter(ax=ax[1], x='P_el [W]',y='P_el_sim', alpha=0.1, figsize=(15,5), grid=True, title='Electrical input power [W]')
data.plot.scatter(ax=ax[2], x='P_th [W]',y='P_th_sim', alpha=0.1, figsize=(15,5), grid=True, title='Thermal output power [W]')
ax[0].set_xlim(0,10)
ax[0].set_ylim(0,10)
ax[1].set_xlim(0,20000)
ax[1].set_ylim(0,20000)
ax[2].set_xlim(0,50000)
ax[2].set_ylim(0,50000)

#### **6.4.2 Cooling** <a class="anchor" id="air-water-regulated-cooling" name="air-water-regulated-cooling"></a>

In [ ]:
data = pd.read_csv('../output/database_cooling_reduced_normalized_validation.csv')
data = data[['T_outside [°C]','RE_Pdc', 'RE_P_el', 'RE_EER']]
ax = data.boxplot(by='T_outside [°C]', layout=(1,3), figsize=(10,5), showfliers=False)
ax[0].set_ylim(-60,60)
ax[0].set_ylabel('relative error in %')
data.abs().mean()[1:4] # mean absolute percentage error (MAPE)

In [ ]:
data = pd.read_csv('../output/database_cooling_reduced_normalized_validation.csv')
fig, ax = plt.subplots(1,3)
data.plot.scatter(ax=ax[0], x='EER',y='EER_sim', alpha=0.3, figsize=(12,5), grid=True, title='Coefficient of performance')
data.plot.scatter(ax=ax[1], x='P_el [W]',y='P_el_sim', alpha=0.1, figsize=(15,5), grid=True, title='Electrical input power [W]')
data.plot.scatter(ax=ax[2], x='Pdc [W]',y='Pdc_sim', alpha=0.1, figsize=(15,5), grid=True, title='Thermal output power [W]')
ax[0].set_xlim(0,10)
ax[0].set_ylim(0,10)
ax[1].set_xlim(0,6000)
ax[1].set_ylim(0,6000)
ax[2].set_xlim(0,15000)
ax[2].set_ylim(0,15000)

### **6.5 Brine/Water | regulated** <a class="anchor" id="brine-water-regulated" name="brine-water-regulated"></a>

* The mean absolute percentage error (MAPE) is over all heat pumps is
    * 4.2 % for COP
    * 17.7 % for P_el
    * 19.7 % for P_th
* **Important**: Validation data is only available at primary input temperature of 0 °C.  

In [ ]:
# Plot relative error for all heat pumps of type brine/water | regulated
Group = 2
data = df.loc[(df['Group']==Group)]
data = data[['T_amb [°C]','RE_COP', 'RE_P_el', 'RE_P_th']]
ax = data.boxplot(by='T_amb [°C]', layout=(1,3), figsize=(10,5), showfliers=False)
ax[0].set_ylim(-60,60)
ax[0].set_ylabel('relative error in %')
data.abs().mean()[1:4] # mean absolute percentage error (MAPE)

In [ ]:
# Plot absolute values all heat pumps of type brine/water | regulated as scatter plot
Group = 2
fig, ax = plt.subplots(1,3)
data = df.loc[(df['Group']==Group)]
data.plot.scatter(ax=ax[0], x='COP',y='COP_sim', alpha=0.3, figsize=(12,5), grid=True, title='Coefficient of performance')
data.plot.scatter(ax=ax[1], x='P_el [W]',y='P_el_sim', alpha=0.1, figsize=(15,5), grid=True, title='Electrical input power [W]')
data.plot.scatter(ax=ax[2], x='P_th [W]',y='P_th_sim', alpha=0.1, figsize=(15,5), grid=True, title='Thermal output power [W]')
ax[0].set_xlim(0,10)
ax[0].set_ylim(0,10)
ax[1].set_xlim(0,20000)
ax[1].set_ylim(0,20000)
ax[2].set_xlim(0,50000)
ax[2].set_ylim(0,50000)

## **7. Conclusion** <a class="anchor" id="conclusion" name="conclusion"></a>

- On/Off heat pumps can be simulated very well (mean relative error < 6 %)
- Regulated heat pumps show a good values in terms of COP and EER but relative mean errors about 15-20 % for electrical and thermal power, because of the non-linearity regarding different primary / secondary temperatures.
- Despite of that, generic heat pumps should work well, because the median shows only a small relative error

## **8. Additional functions** <a class="anchor" id="additional-functions" name="additional-functions"></a>

The hplib has an additonal class with functions regarding the heating system. For example it is possible to calculate the brine temperatur or the calculate the temperature of the heating distribution system.

In [ ]:
# Initialize a heating system
t_hs_set = [35,28]  # example for floor heating
f_hs_exp = 1.1      # example for floor heating
t_min = -11         # example minimum reference outside temperature for building

HS = hpl.HeatingSystem(t_outside_min=-11,t_hs_set=t_hs_set,f_hs_exp=f_hs_exp)

In [ ]:
# Calculate heating flow and return temperature
t_avg_d = -3        # average temperatur of day

t_dist = HS.calc_heating_dist_temp(t_avg_d=t_avg_d)
print('Heating flow temperatur = '+str(round(t_dist[0],1)))
print('Heating return temperatur = '+str(round(t_dist[1],1)))

In [ ]:
# Calculate brine temperature as function of outside temperature
# for heat pumps with earth probes
t_avg_d = -3

t_brine = HS.calc_brine_temp(t_avg_d = t_avg_d)
print('Brine flow temperatur = '+str(round(t_brine,1)))
